In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
df1 = pd.read_csv('../../magentacloud/output_Marktplatz-20180410-141306.csv')
df1 = df1[df1['w']<100] # filter large boxes

In [20]:
df1['distance'] = df1.groupby(['track_id']).diff().dropna().apply(
    lambda row : np.sqrt((row['x']*.026)**2 + (row['y']*.037)**2), axis = 1)
df1['frame_diff'] = df1.groupby(['track_id'])[['frame_id']].diff()

In [21]:
df1['velocity'] = df1['distance'] / ( df1['frame_diff'] * (1.0/30))
df1['velocity_smooth'] = df1.groupby(['track_id'])['velocity'].apply(pd.rolling_mean, 5, min_periods=1)

/home/florian/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:796: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,min_periods=1,center=False).mean()
  return func(g, *args, **kwargs)


In [22]:
counts = df1.groupby(['frame_id']).count().reset_index()[['frame_id', 'track_id']].rename({'track_id':'count'},axis=1)

In [23]:
df2 = pd.merge(df1,counts,on='frame_id')

In [13]:
df2 = df2.sort_values(['frame_id','velocity_smooth'], ascending=False).drop_duplicates(['frame_id']).sort_values('frame_id')

In [28]:
json = df2[['frame_id', 'track_id', 'velocity_smooth', 'count']].to_json(orient='records')

In [29]:
with open("json2.txt", "w") as text_file:
    text_file.write(json)